In [ ]:
# ! pip install torchtext==0.6.0 pyvi spacy https://gitlab.com/trungtv/vi_spacy/-/raw/master/vi_core_news_lg/dist/vi_core_news_lg-0.0.1.tar.gz rouge_score
# ! python -m spacy download en_core_web_sm

In [ ]:
# ! git clone https://github.com/namnh194/transformer_pointer_generator_network.git

In [2]:
# refer: github.com/pbcquoc
from dataset import read_data, create_fields, create_dataset
from model import Transformer
from train_utils import ScheduledOptim, LabelSmoothingLoss, step, validiate
from inference import rouge_score, translate_sentence
import torch, tqdm
import torch.nn as nn
import torch.nn.functional as F

C:\Users\Admin\Desktop\transformer_pointer_generator_network\inference.py:159: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = datasets.load_metric("rouge")


In [3]:
import datasets

dataset = datasets.load_dataset("nam194/vietnews")
dataset

DatasetDict({
    test: Dataset({
        features: ['guid', 'title', 'abstract', 'article'],
        num_rows: 22498
    })
    train: Dataset({
        features: ['guid', 'title', 'abstract', 'article'],
        num_rows: 99134
    })
    validation: Dataset({
        features: ['guid', 'title', 'abstract', 'article'],
        num_rows: 22184
    })
})

In [6]:
opt = {
    'lang': 'vi_core_news_lg',#'vi_spacy_model',
    'en_max_strlen': 256,
    'de_max_strlen': 64,
    'batchsize': 64,
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    'd_model': 512,
    'n_layers': 6,
    'heads': 8,
    'dropout': 0.1,
    'lr': 0.0001,
    'epochs': 20,
    'printevery': 200,
    'k': 5,
}

In [7]:
train_src_data, train_trg_data = read_data(dataset, 'train', 'article', 'abstract')
valid_src_data, valid_trg_data = read_data(dataset, 'validation', 'article', 'abstract')

tokenizer = create_fields(opt['lang'])
train_iter = create_dataset(train_src_data[:500], train_trg_data[:500], opt['batchsize'], opt['device'], tokenizer, istrain=True)
valid_iter = create_dataset(valid_src_data[:100], valid_trg_data[:100], opt['batchsize'], opt['device'], tokenizer, istrain=False)

loading spacy tokenizers...
creating dataset and iterator... 
creating dataset and iterator... 


In [8]:
src_pad = tokenizer.vocab.stoi['<pad>']
trg_pad = tokenizer.vocab.stoi['<pad>']
src_pad, trg_pad

(1, 1)

In [ ]:
from train_utils import create_masks

src = batch.src.transpose(0, 1)
trg = batch.trg.transpose(0, 1)
trg_input = trg[:, :-1]
src_mask, trg_mask = create_masks(src, trg_input, src_pad, trg_pad, device='cpu')
# preds = model(src, trg_input, src_mask, trg_mask)
print(src.shape, trg.shape, trg_input.shape)
print(src_mask.shape, trg_mask.shape, src_mask, trg_mask)

In [10]:
en_config = {
    "max_seq_len": opt['en_max_strlen'],
    "d_model": opt['d_model'],
    "n_heads": opt['heads'],
    "d_ff": 2048,
    "num_layer": opt['n_layers'],
    "dropout": opt['dropout']}
de_config = {
    "max_seq_len": opt['de_max_strlen'],
    "d_model": opt['d_model'],
    "n_heads": opt['heads'],
    "d_ff": 2048,
    "num_layer": opt['n_layers'],
    "dropout": opt['dropout']}

vocab_size = 100 # len(tokenizer.vocab)
batch_size = opt['batchsize']
en_seq_len = en_config["max_seq_len"]
de_seq_len = de_config["max_seq_len"]

model = Transformer(vocab_size, en_config, de_config)
print(sum(p.numel() for p in model.parameters() if p.requires_grad))

for p in model.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)
model = model.to(opt['device'])

44243044


In [ ]:
import gc
from accelerate import Accelerator

optimizer = ScheduledOptim(
        torch.optim.Adam(model.parameters(), betas=(0.9, 0.98), eps=1e-09),
        0.2, opt['d_model'], 4000)
criterion = LabelSmoothingLoss(len(TRG.vocab), padding_idx=trg_pad, smoothing=0.1)

model, optimizer, train_iter, valid_iter = Accelerator().prepare(
    model, optimizer, train_iter, valid_iter)

torch.cuda.empty_cache()
gc.collect()

In [ ]:
!wandb login e67e803979133f932a24dac4dedb24348671c1ba

In [ ]:
import wandb
wandb.init(
    # set the wandb project where this run will be logged
    project="transformer_VNDS",
    
    # track hyperparameters and run metadata
    config=opt
)

In [ ]:
import time

for epoch in range(opt['epochs']):
    total_loss = 0
    
    for i, batch in enumerate(train_iter): 
        s = time.time()
        loss = step(model, optimizer, batch, criterion, src_pad, trg_pad)
        
        total_loss += loss
        
        if (i + 1) % opt['printevery'] == 0:
            avg_loss = total_loss/opt['printevery']
            wandb.log({"train_loss": avg_loss, "step": (i+1)+epoch*3097})
            print('epoch: {:03d} - iter: {:05d} - train loss: {:.4f} - time: {:.4f}'.format(epoch, i, avg_loss, time.time()- s))
            total_loss = 0
            
    s = time.time()
    valid_loss = validiate(model, valid_iter, criterion, src_pad, trg_pad)
    score = rouge_score(valid_src_data[:500], valid_trg_data[:500], model, SRC, TRG, opt['device'], opt['k'], opt['max_strlen'])
    print('epoch: {:03d} - iter: {:05d} - valid loss: {:.4f} - bleu score: {:.4f} - time: {:.4f}'.format(epoch, i, valid_loss, score, time.time() - s))

In [139]:
batch = next(iter(valid_iter)).src[:,0]
print(batch.shape)
sentence = []

for i in batch:
    if i not in [0,1]:
        sentence += [SRC.vocab.itos[i]]
sentence = ' '.join(sentence).replace('_',' ')
print(sentence)

trans_sent = translate_sentence(sentence, model, SRC, TRG, opt['device'], opt['k'], opt['de_max_strlen'])
trans_sent

torch.Size([256])
Khuôn viên Đại học Stanford , bang California , Mỹ . Các nguồn tin giấu tên hôm qua tiết lộ một gia đình Trung Quốc đã trả 6,5 triệu USD cho William Rick Singer , chủ sở hữu công ty tư vấn đào tạo dự bị đại học Edge College & Career Network , để đảm bảo cho con gái của họ có một suất tại Đại học Stanford , Mỹ . Đây được cho là khoản tiền lớn nhất mà các phụ huynh trả cho Singer , kẻ cầm đầu đường dây chạy vào các trường đại học danh giá của Mỹ . Sau Chiến dịch Versity Blues kéo dài một năm , các công tố viên liên bang Mỹ hồi tháng 3 công bố cách thức hoạt động của đường dây này , đồng thời nộp hồ sơ truy tố lên Bộ Tư pháp . Truyền thông Mỹ xác định nữ sinh Trung Quốc được bố mẹ chi tiền chạy vào Đại học Stanford có tên là Yusi " Molly " Zhao . Gia đình Zhao sống tại Bắc Kinh và được một cố vấn tài chính của ngân hàng Morgan Stanley giới thiệu với Singer . Nữ sinh này được nhận vào Stanford năm 2017 nhờ được Singer giới thiệu là một vận động viên đua thuyền đầy hứa hẹn

'chứng_thư chứng_thư chứng_thư sinh_thời sinh_thời sinh_thời sinh_thời sinh_thời sinh_thời sinh_thời sinh_thời sinh_thời sinh_thời sinh_thời chứng_thư chứng_thư chứng_thư chứng_thư chứng_thư chứng_thư chứng_thư chứng_thư chứng_thư chứng_thư chứng_thư chứng_thư chứng_thư chứng_thư chứng_thư chứng_thư chứng_thư chứng_thư Phúc_Cường sinh_thời Phúc_Cường sinh_thời chứng_thư chứng_thư chứng_thư sinh_thời Phúc_Cường Phúc_Cường Phúc_Cường Vương_Chí_Dân chứng_thư chứng_thư chứng_thư chứng_thư chứng_thư chứng_thư chứng_thư chứng_thư chứng_thư chứng_thư chứng_thư chứng_thư chứng_thư chứng_thư chứng_thư chứng_thư chứng_thư chứng_thư'

0 torch.Size([256, 11])


In [63]:
for i, batch in enumerate(train_iter):
    if batch.src.shape[-1] != 1:
        print(i, batch.src.shape)
        break

for idx in range(batch.src.shape[-1]):
    print('idx: ',idx)
    sentence = []
    for i in batch.src[:,idx]:
        #if i not in [0,1]:
            sentence += [tokenizer.vocab.itos[i].replace('_',' ')]
    print(' '.join(sentence))
    print('---------------------------------------------------')

idx:  0
<sos> Ngày 3 / 7 , thông tin từ đội hiệp sĩ Nguyễn Thanh Hải cho biết , đội vừa nhận được đơn trình báo của tập thể công nhân lao động , tố cáo về việc bị đối tượng Nguyễn Hoàng Quân ( không rõ lai lịch ) lừa đảo chiếm đoạt tiền lương hơn 50 triệu đồng . Đây là tiền công làm thời vụ của cả tập thể . Theo đó , nhóm tập thể công nhân hơn 20 người cho biết , trước đó ngày 24 / 6 , nhóm công nhân này đến làm thời vụ cho công ty may K.T . tại tỉnh lộ 15 , tổ 4 , ấp 11 , xã Tân Thạnh Đông , huyện Củ Chi , TP. HCM với mức lương 300 nghìn đồng / ngày . Tuy nhiên , hợp đồng lao động thời vụ này được thoả thuận bằng miệng trực tiếp với đối tượng Quân , thông qua việc đối tượng này đăng thông tin tuyển lao động trên mạng xã hội . Sau khi tuyển được lao động , Quân trực tiếp đưa công nhân đến công ty K.T . làm việc và thống nhất đến mỗi thứ Bảy hàng tuần sẽ thanh toán tiền công . Đến ngày 29 / 6 , đến thời hạn trả lương cho công nhân thì đối tượng Quân bỏ trốn . Toàn bộ tiền công lao động 

In [77]:
_ = tokenizer.tokenize('Ngày 3 / 7 , thông tin từ đội hiệp sĩ Nguyễn Thanh Hải cho biết , đội vừa nhận được đơn trình báo của tập thể công nhân lao động , tố cáo về việc bị đối tượng Nguyễn Hoàng Quân ( không rõ lai lịch ) lừa đảo chiếm đoạt tiền lương hơn 50 triệu đồng . Đây là tiền công làm thời vụ của cả tập thể . Theo đó , nhóm tập thể công nhân hơn 20 người cho biết , trước đó ngày 24 / 6 , nhóm công nhân này đến làm thời vụ cho công ty may K.T . tại tỉnh lộ 15 , tổ 4 , ấp 11 , xã Tân Thạnh Đông , huyện Củ Chi , TP. HCM với mức lương 300 nghìn đồng / ngày . Tuy nhiên , hợp đồng lao động thời vụ này được thoả thuận bằng miệng trực tiếp với đối tượng Quân , thông qua việc đối tượng này đăng thông tin tuyển lao động trên mạng xã hội . Sau khi tuyển được lao động , Quân trực tiếp đưa công nhân đến công ty K.T . làm việc và thống nhất đến mỗi thứ Bảy hàng tuần sẽ thanh toán tiền công . Đến ngày 29 / 6 , đến thời hạn trả lương cho công nhân thì đối tượng Quân bỏ trốn . Toàn bộ tiền công lao động công nhân đã thực hiện ( trên 50 triệu đồng ) không được thanh toán . Về phía công ty may K.T . , ông Phạm Tuấn C . , Giám đốc công ty cho hay , hợp đồng lao động công ty làm việc trực tiếp với Quân . Tiền công trả cho người lao động ông C. đã thanh toán đầy đủ cho Quân . Còn việc Quân không trả tiền cho công nhân công ty không hề hay biết . Theo trình báo của tập thể người lao động bị chiếm đoạt tiền công cho biết , sau khi Quân nhận tiền từ vợ chồng chủ công ty may , Quân cầm xấp tiền chụp hình đăng lên mạng Zalo , với mục đích khoe khoang và thách thức nên họ càng bức xức hơn . Tập thể công nhân nói trên và Giám đốc công ty may K.T . đã viết đơn trình báo Công an xã Tân Thạnh Đông và đội hiệp sĩ Nguyến Thanh Hải để được giải quyết . Hiện , cơ quan công an và đội hiệp sĩ đang truy tìm đối tượng , điều tra làm rõ vụ việc . Đối tượng Nguyễn Hoàng Quân chiếm đoạt tiền lương của công nhân rồi chụp hình đăng Zalo . Tập thể công nhân đến gặp đội hiệp sĩ Nguyễn Thanh Hải trình báo vụ việc')
print(_)

['Ngày', '3', '/', '7', ',', 'thông tin', 'từ', 'đội', 'hiệp sĩ', 'Nguyễn Thanh Hải', 'cho', 'biết', ',', 'đội', 'vừa', 'nhận', 'được', 'đơn', 'trình báo', 'của', 'tập thể', 'công nhân', 'lao động', ',', 'tố cáo', 'về', 'việc', 'bị', 'đối tượng', 'Nguyễn Hoàng Quân', '(', 'không', 'rõ', 'lai lịch', ')', 'lừa đảo', 'chiếm đoạt', 'tiền lương', 'hơn', '50', 'triệu', 'đồng', '.', 'Đây', 'là', 'tiền', 'công', 'làm', 'thời vụ', 'của', 'cả', 'tập thể', '.', 'Theo', 'đó', ',', 'nhóm', 'tập thể', 'công nhân', 'hơn', '20', 'người', 'cho', 'biết', ',', 'trước', 'đó', 'ngày', '24', '/', '6', ',', 'nhóm', 'công nhân', 'này', 'đến', 'làm', 'thời vụ', 'cho', 'công ty', 'may', 'K.T', '.', 'tại', 'tỉnh lộ', '15', ',', 'tổ', '4', ',', 'ấp', '11', ',', 'xã', 'Tân Thạnh Đông', ',', 'huyện', 'Củ Chi', ',', 'TP.', 'HCM', 'với', 'mức', 'lương', '300', 'nghìn', 'đồng', '/', 'ngày', '.', 'Tuy nhiên', ',', 'hợp đồng', 'lao động', 'thời vụ', 'này', 'được', 'thoả thuận', 'bằng', 'miệng', 'trực tiếp', 'với', 'đối 

In [82]:
tokenizer.vocab.stoi['lao_động']

663